구글드라이브 마운트

In [5]:
import os
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

영상에서 얼굴 인식 후 사이즈에 맞게 자르기

In [8]:
import cv2
import dlib
import os
from moviepy.editor import *

# 얼굴 인식기 초기화
detector = dlib.get_frontal_face_detector()

# 폴더 경로 설정
folder_path = '/Users/hwangjong-won/Desktop/DL/testset/_D3HIqX1OVk/' # 크롭할 파일들이 모여있는 경로입니당

# 출력 동영상 경로 설정
output_folder = '/Users/hwangjong-won/Desktop/DL/testset/facecrop/' # 크롭하고 난 파일들을 따로 한번에 모아둘 경로입니당~
os.makedirs(output_folder, exist_ok=True)

# 폴더 내의 모든 동영상 파일에 대해 처리
for folder_name in os.listdir(folder_path):
    folder_dir = os.path.join(folder_path, folder_name)
    if not os.path.isdir(folder_dir):
        continue

    for filename in os.listdir(folder_dir):
        if not filename.endswith('.mp4'):
            continue

        video_path = os.path.join(folder_dir, filename)
        cap = cv2.VideoCapture(video_path)

        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        # 출력 동영상 설정
        out_filename = f'{filename}.mp4'   # 출력 동영상 이름 입니당
        out_temp_filename = 'temp.mp4'  # 예비 파일이니 파일명 신경 xx

        out = cv2.VideoWriter(out_temp_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

        # 오디오 파일 추출
        audio = AudioFileClip(video_path)

        while True:
            # 프레임 읽기
            ret, frame = cap.read()

            if not ret:
                break

            # 얼굴 인식
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = detector(gray)

            # 얼굴이 인식되지 않은 경우, 전체 영상을 출력
            if len(faces) == 0:
                out.write(frame)
                continue

            # 중앙 얼굴 찾기
            center_x = width // 2
            center_y = height // 2

            min_distance = float('inf')  # 최소 거리 초기화
            selected_face = None

            for face in faces:
                # 얼굴의 중심 좌표 계산
                face_center_x = face.left() + face.width() // 2
                face_center_y = face.top() + face.height() // 2

                # 중심 좌표와의 거리 계산
                distance = ((center_x - face_center_x) ** 2) + ((center_y - face_center_y) ** 2)

                if distance < min_distance:
                    min_distance = distance
                    selected_face = face

            if selected_face is None:
                # 얼굴이 인식되지 않은 경우,  pass
                # out.write(frame)
                continue

            # 중앙 얼굴 영역 잘라내기
            x, y, w, h = selected_face.left(), selected_face.top(), selected_face.width(), selected_face.height()
            x -= 40  # 좌측 여유 공간 추가
            y -= 40  # 상단 여유 공간 추가
            w += 60  # 우측 여유 공간 추가
            h += 60  # 하단 여유 공간 추가

            try:
                # 잘라낸 얼굴 영역 출력
                face_img = frame[y:y+h, x:x+w]

                # 원본 크기로 사이즈 늘리기
                face_resized = cv2.resize(face_img, (width, height))

                # 출력 동영상에 추가
                out.write(face_resized)
            except cv2.error:
                # 이미지 크기 조정 오류 발생 시, pass하고 다음 프레임으로 넘어감
                pass

        # 동영상 종료
        cap.release()
        out.release()

        # # 임시 출력 동영상과 오디오 합성
        # video_temp = VideoFileClip(out_temp_filename)
        # final_video = video_temp.set_audio(audio)

        # # 경로에 저장
        # output_path = os.path.join(output_folder, out_filename)
        # final_video.write_videofile(output_path, codec='libx264')

        # # 임시 파일 삭제
        # video_temp.close()
        # os.remove(out_temp_filename)
        
        
        try:
            video_temp = VideoFileClip(out_temp_filename)
            final_video = video_temp.set_audio(audio)

            # 경로에 저장
            output_path = os.path.join(output_folder, out_filename)
            final_video.write_videofile(output_path, codec='libx264')
        except OSError as e:
            print(f"An error occurred while processing the video: {e}")
        finally:
            # 임시 파일 삭제
            if os.path.exists(out_temp_filename):
                os.remove(out_temp_filename)



